# Features from the paper

`Easley, D., López de Prado, M., O’Hara, M., Zhang, Z., 2020. Microstructure in the Machine Age. Rev. Financ. Stud. 34, 3316–3363`

In [1]:
# import some packages
import numpy as np
import pandas as pd
import os
from pathlib import Path

home = str(Path.home())

In [2]:
# Load the data
data_folder = os.path.join(home, 'data', 'optiver-realized-volatility-prediction')

# for the prototype load just one stock
df_book = pd.read_parquet(os.path.join(data_folder, 'book_train.parquet', 'stock_id=0'))
df_trades = pd.read_parquet(os.path.join(data_folder, 'trade_train.parquet', 'stock_id=0'))
# df_trades

## Features

### 0. Some simple stats

In [3]:
# quoted spread
df_book['spread'] = df_book['ask_price1'] - df_book['bid_price1']


### 1. Roll measure

$R_\tau = 2\sqrt{\text{Cov} (\Delta P_\tau, \Delta P_{\tau-1})}$

### 2. Roll Impact
$\tilde{R}_\tau = \frac{R_\tau}{p_\tau V_\tau}$

### 3. Kyle's lambda (Market Impact)

$\lambda = \frac{| \Delta \text{Price}_t |}{\text{Volume}_t}$

In [20]:
df_measures = df_trades.copy()
df_measures['d_price'] = df_measures.groupby(['time_id'])['price'].diff()
df_measures['d_price_l1'] = df_measures.groupby(['time_id'])['d_price'].shift(1)

grp = df_measures.groupby('time_id')
df_measures = pd.DataFrame()
df_measures['covariance'] = grp.apply(lambda x: x['d_price'].cov(x['d_price_l1']))

# Roll measure
df_measures['roll_measure'] = 2 * np.sqrt(np.abs(df_measures.covariance))
df_measures.drop(columns = ['covariance'], inplace=True)

# Roll impact
df_measures['roll_impact'] = df_measures['roll_measure'] / grp.apply(lambda x: np.sum(x['price'] * x['size']))

# Kyle's lambda
df_measures['mkt_impact'] = grp.apply(lambda x: np.sum(np.abs(x['d_price'])) / np.sum(np.abs(x['size'])))

# df_measures.roll_measure.describe()

df_measures

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/nanops.py:1516: RuntimeWarning: Degrees of freedom <= 0 for slice
  return np.cov(a, b, ddof=ddof)[0, 1]
/opt/anaconda3/lib/python3.8/site-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


,roll_measure,roll_impact,mkt_impact
time_id,,,
5,0.000204,6.382619e-08,0.000003
11,0.000206,1.597455e-07,0.000003
16,0.000457,2.118416e-07,0.000004
31,0.000516,2.630821e-07,0.000002
62,0.000065,3.642724e-08,0.000002
...,...,...,...
32751,0.000207,6.012802e-08,0.000002
32753,0.000239,5.259393e-08,0.000001
32758,0.000248,5.843691e-08,0.000002


### 4.